# Credit VAR of a Swap

The goal of the project is to compute the Credit VAR (99.9th percentile) of a swap in which the counterparty might default. The input parameters are:

* the par rate of a set of Overnight Index Swaps (i.e. their market quotation);
* a set of forward libor rates and their fixing dates;
* the static data of the swap (nominal, start date, maturity, underlying libor tenor, etc..). The fixed rate of the swap is such that if there was no counterparty risk, the product would be traded at par. You are requested to compute it;
* a set of survival probabilities for the risky counterparty;
* the recovery rate for this swap (i.e. the fraction of the market value that it is expected to recover in case of default);
* the volatility of the swap rate, which is assumed to be constant through time (the swap rate is lognormal with fixed volatility).

The swap starts today and the horizon for the VAR calculation is before any future payments. 

## Notes and Hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for credit risk evaluation.

Please try to reuse as much as possible the code in $\texttt{finmarkets.py}$, and only when it is not viable write your own classes.


## Inputs

[Overnight swap quotes](https://github.com/matteosan1/finance_course/raw/develop/Projects/ois_quotes.xlsx)

[Counterparty survival probabilities](https://raw.githubusercontent.com/matteosan1/finance_course/develop/Projects/survival_probabilities_proj3.csv)

[Euribor 6M rates (choose right sheet)](https://github.com/matteosan1/finance_course/raw/develop/Projects/euribor_curve.xlsx)

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

irs_to_price = {
    'nominal': 100e6, 
    'libor_tenor_months': 6,
    'fixed_leg_frequency': 6,
    'maturity_months': 72
}

pricing_date = date.today()
credit_var_date = pricing_date + relativedelta(years=2)
recovery = 0.4
volatility = 0.4
mc_runs = 100000
percentile = 0.999